In [138]:
#imports
import requests
import pandas as pd
import os
import datetime as dt
from google.cloud import bigquery
from google.cloud import storage
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np
import meteostat as ms

In [139]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [140]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [141]:
timestamp = dt.datetime.now()
timestamp = timestamp.strftime("%Y%m%d")

In [142]:
loglist = []

In [143]:
def getmaxdate(bq_client):
    QUERY = (
        'SELECT MAX(date) FROM `capstone-team51.chicago_data_prod.consolidated_table` '
        )
    query_job = bq_client.query(QUERY)  # API request
    df = query_job.to_dataframe()  # Waits for query to finish
    maxdate = df.iloc[0,0].to_pydatetime().date().strftime("%Y-%m-%d")
    return maxdate


In [144]:
#primary data pull function

def get_data(name, db, maxrows, parameters, loglist, bucket, filepath):
    # some key info
    db_id = db
    limit = maxrows
    i = 0
    offset_counter = 0
    pickle = True
    data_name = name
    # dict for logging
    log_dict = {}

    # base url for the request
    baseurl = ("https://data.cityofchicago.org/resource/"
            f"{db_id}.json?")

    while pickle:
        #set our params
        # note that you have to URL econde 
        params = parameters + (
            f"&$limit={limit}"
            f"&$offset={offset_counter}"
            #f"&$$app_token={token}"
        )
        
        # make the request 
        data_req = requests.get(baseurl+params)

        # convert to json
        data_json = data_req.json()

        # make it a df
        df = pd.json_normalize(data_json)

        # write df
        
        
        # log
        minilog = {}
        minilog['name'] = name
        minilog['records'] = df.shape[0]
        minilog['chunk'] = i
        minilog['status'] = data_req.status_code
        minilog['offset'] = offset_counter
        minilog['date'] = timestamp
        
        
        if df.shape[0] == 0:
            break
        else:
            bucket.blob(f'{filepath}/{timestamp}_{data_name}_chunk_{i}.csv').upload_from_string(df.to_csv(index=False), 'text/csv')
            #df.to_json(f'{desti_folder}/{timestamp}_{data_name}_chunk_{i}.json',orient='records', lines=True)
            loglist.append(minilog)

        # increment the chunk count
        print(i)
        i += 1

        # increment the offset
        offset_counter += limit

        # check if we need to end the loop
        if df.shape[0] == 0:
            pickle = False
        elif data_req.status_code == 200:
            pickle = True
        else: 
            pickle = False
            print(data_req.status_code)

In [156]:
dt.datetime.now().date()

datetime.date(2024, 2, 4)

In [152]:
def get_weather_data(name, start_date, minilog, bucket, filepath):
    ## weather
    start = dt.datetime.strptime(start_date, '%Y-%m-%d').date()
    end = dt.datetime.now().date()
    # Create Point for chicago
    chicago = ms.Point(41.85, -87.65)

    dailyholder = ms.Daily(chicago, start, end)
    data_daily = dailyholder.fetch()
    data_daily[['tempmax_f', 'tempmin_f', 'tempavg_f']] = data_daily[['tmax', 'tmin', 'tavg']].apply(lambda x: (x * 9/5) + 32)
    bucket.blob(f'{filepath}/{timestamp}_weather_daily_data.csv').upload_from_string(data_daily.to_csv(), 'text/csv')
    #data_daily.to_csv(f'./data/{timestamp}_weather_daily_data.csv')

    minilog = {}
    minilog['name'] = name
    minilog['records'] = data_daily.shape[0]
    minilog['chunk'] = 0
    minilog['status'] = 200
    minilog['offset'] = 0
    minilog['date'] = timestamp
    #log_df = pd.DataFrame.from_dict(minilog,orient='index')
    loglist.append(minilog)

In [159]:
dt.datetime.strptime(start_date, '%Y-%m-%d').date()

datetime.date(2023, 12, 31)

In [160]:
dt.datetime.now().date()

datetime.date(2024, 2, 4)

In [158]:
start = dt.datetime.strptime(start_date, '%Y-%m-%d').date()
end = dt.datetime.now().date()
# Create Point for chicago
chicago = ms.Point(41.85, -87.65)

dailyholder = ms.Daily(chicago, start, end)
data_daily = dailyholder.fetch()
data_daily[['tempmax_f', 'tempmin_f', 'tempavg_f']] = data_daily[['tmax', 'tmin', 'tavg']].apply(lambda x: (x * 9/5) + 32)
bucket.blob(f'{filepath}/{timestamp}_weather_daily_data.csv').upload_from_string(data_daily.to_csv(), 'text/csv')
#data_daily.to_csv(f'./data/{timestamp}_weather_daily_data.csv')

minilog = {}
minilog['name'] = name
minilog['records'] = data_daily.shape[0]
minilog['chunk'] = 0
minilog['status'] = 200
minilog['offset'] = 0
minilog['date'] = timestamp
#log_df = pd.DataFrame.from_dict(minilog,orient='index')
loglist.append(minilog)

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'datetime.date'

## Params and pulls

In [146]:
start_date = getmaxdate(bq_client)

In [147]:
# crime
crime_params = (
            f"$where=DATE%20%3E%3D%20%27{start_date}%27"
            f"&$order=DATE"
            #f"&$$app_token={token}"
            )

get_data('crime', 'ijzp-q8t2', 500000, crime_params, loglist, bucket, 'raw_crimes')

0


In [148]:
# dph env complaints
env_params = (
            f"$where=complaint_date%20%3E%3D%20%27{start_date}%27"
            f"&$order=complaint_date"
            #f"&$$app_token={token}"
            )

get_data('dph_env', 'fypr-ksnz', 500000, env_params, loglist, bucket, 'raw_environmental')

0


In [149]:
# 311 complaints
three11_params = (
            f"$where=(created_date%20%3E%3D%20%27{start_date}%27)%20AND%20sr_short_code%20not%20in%20(%27311IOC%27)"
            f"&$order=created_date"
            )

get_data('311', 'v6vf-nfxy', 500000, three11_params, loglist, bucket, 'raw_311')



0


In [153]:
# weather data
get_weather_data('weather', start_date, loglist, bucket, 'raw_weather')

In [151]:
log_df = pd.DataFrame(loglist)
bucket.blob(f'logs/{timestamp}_pull_log.csv').upload_from_string(log_df.to_csv(index=False), 'text/csv')